In [21]:
## USER 테이블 build
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = (
    SparkSession.builder \
                .master("local")
                .appName("World Count")
                .getOrCreate()
)
file_path = r"C:\Users\brian\Desktop\JUNSOO\Project\ETL_DATA\data\ranking_2024-10-11.json"
df = spark.read.format('json').option("multiLine", True).load(file_path)

# 직업명을 제대로 채우기
import pyspark.sql.functions as F
df = df.select(F.explode("ranking").alias("USER"))
df = df.select("USER.character_name",
               "USER.date",
               "USER.class_name",
               "USER.sub_class_name",
               "USER.character_level",
               "USER.character_exp",
               "USER.ranking")
df = df.withColumn("class",df["sub_class_name"])
df = df.withColumn("class",F.when(df["sub_class_name"]== "", df["class_name"]) \
                            .otherwise(df["class"]))
df = df.drop("class_name","sub_class_name")


# 각 유저의 위치한 지역 컬럼추가
df = df.withColumn("status",
                   F.when(df["character_level"]>=290,"Tallahart") \
                    .when((df["character_level"]<=289)&(df["character_level"]>=285),"Carcion") \
                    .when((df["character_level"]<=284)&(df["character_level"]>=280),"Arteria") \
                    .when((df["character_level"]<=279)&(df["character_level"]>=275),"Dowonkyung") \
                    .when((df["character_level"]<=274)&(df["character_level"]>=270),"Odium") \
                    .when((df["character_level"]<=269)&(df["character_level"]>=265),"HotelArcs") \
                    .when((df["character_level"]<=264)&(df["character_level"]>=260),"Cernium") \
                    .otherwise("AcaneRiver"))

df.show(10)

+--------------+----------+---------------+---------------+-------+-----------------+---------+
|character_name|      date|character_level|  character_exp|ranking|            class|   status|
+--------------+----------+---------------+---------------+-------+-----------------+---------+
|          버터|2024-10-11|            297|843016259192919|      1|       나이트로드|Tallahart|
|          헨쇼|2024-10-11|            296|759870313066254|      2|             아델|Tallahart|
|   케인WWE챔프|2024-10-11|            296| 24509577197550|      3|       배틀메이지|Tallahart|
|          벨벨|2024-10-11|            295|762723872665559|      4|       스트라이커|Tallahart|
|       검성OGC|2024-10-11|            295|557321451486306|      5|           히어로|Tallahart|
|       Lacheln|2024-10-11|            295|492644225241999|      6|             은월|Tallahart|
|          곤주|2024-10-11|            295|426659239260704|      7|아크메이지(불,독)|Tallahart|
|          광선|2024-10-11|            295|416194201863297|      8|             제논|Ta